# Transfer learning и fine-tuning

Данный блокнот основан на материалах документации фреймворка TensorFlow, опубликованных коллективом TensorFlow Authors: https://www.tensorflow.org/guide/keras/transfer_learning.

In [ ]:
!pip install tensorflow tensorflow_datasets

ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Как осуществить transfer learning?
Процесс transfer learning'а заключается в следующем:
1. Возьмем «backbone» (или другие слои, если вы считаете это необходимым) от модели, натренированной на схожей задаче. Такой процесс иноглда называют «отрыванием головы».
2. «Заморозим» их, то есть сделаем их необучаемыми.
3. Добавим несколько новых обучаемых слоев («голову»), которые будут учиться преобразовывать признаки с уже обученных предыдущих слоев для принятия решения в новой задаче
4. Обучим всю новую сеть (в которой backbone заморожен) на новых данных

# Fine-tuning
После того, как мы обучим новую голову с перенесенным backbone'ом, достаточно часто применяют процесс fine-tuning'а. На предыдущих шагах мы никак не изменяли backbone, а значит в нем присутствуют некоторые признаки, которые либо бесполезны, либо мешают новой задаче. Идея fine-tuning заключается в том, что после того, как мы научили новую голову работать со старым backbone'ом, мы можем «разморозить» всю сеть (реже -- только часть backbone'а) и пообучать ее несколько эпох с очень низким learning rate'ом. Это поможет подстроить старые слои к решению новой задачи, перестроить или адаптировать некоторые фильтры, и сделать процесс принятия решения более цельным. Низкий learning rate и факт предобучения головы со старым backbone'ом позволяет не развалить всю сеть, а аккуратно подстроить ее под нужную область.

# Заморозка весов
Слои нейронной сети состоят из весов, большая часть из которых является тренируемыми (хотя некоторые - нет, так как нужны для определенных операций).
Давайте посмотрим на веса простого полносвязного слоя:

In [ ]:
layer = keras.layers.Dense(3)
layer.build((None, 4))  # Инициализируем веса

print("weights:", len(layer.weights))
print("trainable_weights:", len(layer.trainable_weights))
print("non_trainable_weights:", len(layer.non_trainable_weights))

Metal device set to: Apple M1 Pro
weights: 2
trainable_weights: 2
non_trainable_weights: 0


2022-07-23 16:39:59.128614: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-23 16:39:59.129245: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Вы можете изменить атрибут trainable у слоя, чтобы сделать веса нетренируемыми. В процессе тренировки данные веса не будут изменяться.

In [ ]:
layer = keras.layers.Dense(3)
layer.build((None, 4))  # Инициализируем веса
layer.trainable = False  # Заморозим слой

print("weights:", len(layer.weights))
print("trainable_weights:", len(layer.trainable_weights))
print("non_trainable_weights:", len(layer.non_trainable_weights))

weights: 2
trainable_weights: 0
non_trainable_weights: 2


Кроме того, когда вы устанавливаете флаг заморозки на модель, он устанавливается на все объекты и подобъекты данной модели.

In [ ]:
inner_model = keras.Sequential(
    [
        keras.layers.Dense(3, activation="relu"),
        keras.layers.Dense(3, activation="relu"),
    ]
)

model = keras.Sequential(
    [keras.Input(shape=(3,)), inner_model, keras.layers.Dense(3, activation="sigmoid"),]
)

model.trainable = False  # Freeze the outer model

assert inner_model.trainable == False  # Все веса модели заморожены
assert inner_model.layers[0].trainable == False  # и внутренней модели тоже

Давайте скопируем старые веса, попробуем обучить модель чему-нибудь и сравним веса после обучения со старыми.

In [ ]:
old_weights = model.get_weights()

model.compile(optimizer="adam", loss="mse")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)), epochs=3)

for i, (old_w, new_w) in enumerate(zip(old_weights, model.get_weights())):
    print(f"Веса слоев {i} равны: {np.allclose(old_w, new_w)}")

Epoch 1/3


2022-07-23 16:40:21.980533: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-23 16:40:22.113494: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 388ms/step - loss: 0.0750
Epoch 2/3
1/1 [==============================] - 0s 5ms/step - loss: 0.0750
Epoch 3/3
1/1 [==============================] - 0s 4ms/step - loss: 0.0750
Веса слоев 0 равны: True
Веса слоев 1 равны: True
Веса слоев 2 равны: True
Веса слоев 3 равны: True
Веса слоев 4 равны: True
Веса слоев 5 равны: True


Как можно заметить, веса модели не изменились в процессе обучения.

##  Применение tranfer learning'а в фреймворке Keras

В фреймворке Keras есть встроенная возможность использовать пре-тренированные модели.  

Для этого в модуле `keras.applications` представлены классы для инициализации различных моделей. При инициализации нам интересные следующие параметры:
- weights: источник весов. None для случайной инициализации, imagenet для pre-trained модели на наборе данных ImageNet, или путь до весов данной модели
- input_shape или input_tensor: форма входного тензора или ссылка на него
- include_top: включать ли 'голову' (полносвязные слои)

Пример инициализации сети EfficientNet для transfer learning'а:
```python
base_model = keras.applications.EfficientNetB0(
    weights='imagenet',
    input_shape=(150, 150, 3),
    include_top=False
) 
```

Далее заморозим текущую модель:

```python
base_model.trainable = False
```

После чего создадим подходящие входной тензор и 'голову' модели. Заметьте, что в данном случае мы пользуемся "функциональным" стилем создания слоев 

```python
inputs = keras.Input(shape=(150, 150, 3))
x = base_model(inputs, training=False)

# Вместо Flatten() будем использовать GlobalAveragePooling. О разнице можно прочитать, например, здесь: https://stackoverflow.com/questions/49295311/what-is-the-difference-between-flatten-and-globalaveragepooling2d-in-keras
x = keras.layers.GlobalAveragePooling2D()(x)

# Голова модели - несколько (в данном случае - один) полносвязных слоев
outputs = keras.layers.Dense(1, activation='softmax')(x)

# Создадим объект модели, указав входные и выходные тензоры
model = keras.Model(inputs, outputs)
```

После чего скомпилируем модель с необходимыми параметрами и вызовем метод `fit()` на нужным нам данных.

```python
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=[keras.metrics.BinaryAccuracy()])
model.fit(new_dataset, epochs=20, callbacks=..., validation_data=...)
```

## Реализация fine-tuning в фреймворке Keras

После тренировки модели и достижения достаточной точности, можно разморозить слои и провести процесс fine-tuning'а модели.
Для этого флагу `trainable` всей модели снова присваивается значение `True` и, 
как мы указывали ранее, производится обучение с небольшим значением learning rate.

```python
base_model.trainable = True

# После изменения статуса trainable необходима рекомпиляция модели
model.compile(optimizer=keras.optimizers.Adam(1e-5),  # Не забываем про низкий learning rate
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

model.fit(new_dataset, epochs=10, callbacks=..., validation_data=...)
```

## Пример реализации

Давайте в качестве примера выполним все вышеописанное на наборе данных "tf_flowers". Как обычно, скачаем набор данных и импорируем.

In [ ]:
import tensorflow_datasets as tfds

(train_ds, test_ds), ds_info = tfds.load(
    "tf_flowers",
    split=["train[:30%]", "train[30%:40%]"],     # Возьмем набор данных поменьше
    as_supervised=True,
    with_info=True
)

# Задача мультиклассовой классификации
NUM_CLASSES = ds_info.features['label'].num_classes

2022-07-23 16:40:48.154122: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to ~/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


Т.к. изображения представлены разного размера. Приведем их к единому размеру, который используется сетью EfficientNetB0 (224x224) и соберем в батчи по 32 изображения.

In [ ]:
SIZE = (224, 224)
SHAPE = (*SIZE, 3)
BATCH_SIZE = 32

train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, SIZE), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, SIZE), y))

Кроме того, стандартным способом преобразуем метки классов в one-hot вектора

In [ ]:
def make_one_hot(x, y):
    return x, tf.one_hot(y, depth=NUM_CLASSES)

train_ds = train_ds.map(make_one_hot).batch(BATCH_SIZE)
test_ds = test_ds.map(make_one_hot).batch(BATCH_SIZE)

In [ ]:
base_model = keras.applications.EfficientNetB0(
    weights='imagenet',
    input_shape=SHAPE,
    include_top=False
) 

# Заморозим веса
base_model.trainable = False

# Создадим входной тензор
inputs = keras.Input(shape=SHAPE)

x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(128)(x)
outputs = keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)
model = keras.Model(inputs, outputs)

model.summary()

16705208/16705208 [==============================] - 4s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense_5 (Dense)             (None, 128)               163968    
                                                                 
 dense_6 (Dense)             (None, 5)                 645    

## Train the top layer

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.CategoricalAccuracy()],
)

In [ ]:
# Количество эпох выставлено для быстрого обучения на слабых компьютерах
# Если у вас есть графический ускоритель - можно поставить вплоть до 20
epochs = 3  
model.fit(train_ds, epochs=epochs, validation_data=test_ds)

Epoch 1/3


2022-07-23 16:41:50.046754: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


35/35 [==============================] - ETA: 0s - loss: 0.5615 - categorical_accuracy: 0.7856

2022-07-23 16:41:58.559930: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


35/35 [==============================] - 13s 190ms/step - loss: 0.5615 - categorical_accuracy: 0.7856 - val_loss: 0.3538 - val_categorical_accuracy: 0.8692
Epoch 2/3
35/35 [==============================] - 4s 121ms/step - loss: 0.1457 - categorical_accuracy: 0.9591 - val_loss: 0.3279 - val_categorical_accuracy: 0.8747
Epoch 3/3
35/35 [==============================] - 4s 116ms/step - loss: 0.0742 - categorical_accuracy: 0.9882 - val_loss: 0.3075 - val_categorical_accuracy: 0.8856


## Fine-tuning

После обучения мы можем выполнить несколько эпох fine-tuning'а.

Для этого разморозим веса, перекомпилируем модель (веса не сбрасываются при перекомпиляции) 
и обучим модель в течение нескольких эпох с небольшим learning rate.

In [ ]:
base_model.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.CategoricalAccuracy()],
)

epochs = 1
model.fit(train_ds, epochs=epochs, validation_data=test_ds)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense_5 (Dense)             (None, 128)               163968    
                                                                 
 dense_6 (Dense)             (None, 5)                 645       
                                                             

2022-07-23 16:42:12.031283: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


35/35 [==============================] - ETA: 0s - loss: 0.0359 - categorical_accuracy: 0.9973

2022-07-23 16:42:35.086472: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


35/35 [==============================] - 29s 524ms/step - loss: 0.0359 - categorical_accuracy: 0.9973 - val_loss: 0.3152 - val_categorical_accuracy: 0.8883


Мы с вами провели процесс transfer learning'а - переобучения существующей модели к нашей текущей задаче. С помощью данного процесса вы можете использовать большие предобученные модели (в идеале - на схожих датасетах) для ваших задач с небольшими ресурсами для дообучения.

Желаем успехов в дальнейшем обучении!
